In [ ]:
import scipy.signal as signal
import scipy.fftpack as fftpack
from matplotlib.ticker import EngFormatter
import numpy as np
import matplotlib.pyplot as plt

formatter_db = EngFormatter(unit='dB')
formatter_freq = EngFormatter(unit='Hz')
formatter_time = EngFormatter(unit='s')
formatter_V = EngFormatter(unit='V')

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

def decibel(val_array):
    abs_val_array = np.abs(val_array)
    array_max = np.amax(abs_val_array)
    return 20*np.log10(np.divide(abs_val_array, array_max))

In [ ]:
## Sample rate and desired cutoff frequencies (in Hz).
fs = 4000000.0
sim_gear_freq = 5400 #tri wave
sim_signal_freq = 50000 #sine wave
zoom_offset = 1000  

Vpp_sim_signal = 0.0004
Vpp_sim_gear = 0.4


lowcut = sim_signal_freq - 5000
highcut = sim_signal_freq + 5000
# Plot the frequency response for a few different orders.


# Filter a noisy signal.
T = 0.1# tim_lim=0.0003
nsamples = int(T * fs)
t = np.linspace(0, T, nsamples, endpoint=False)


x = 0.5 * Vpp_sim_gear * signal.sawtooth(2 * np.pi * sim_gear_freq * t)
x += 0.5 * Vpp_sim_signal * np.cos(2 * np.pi * sim_signal_freq * t)

freq_data =  fftpack.fftfreq(nsamples) * fs
x_fft = decibel(fftpack.fft(x))
fft_frequencies = fftpack.fftshift(freq_data)
fft_amplitudes = fftpack.fftshift(x_fft)

In [ ]:
fig, ax = plt.subplots(nrows=3,figsize=(12,6))
ax[0].plot(t,0.5 * Vpp_sim_gear * signal.sawtooth(2 * np.pi * sim_gear_freq * t),'yellow',label='Gearmesh Signal')
ax[0].yaxis.set_major_formatter(formatter_V)
ax[0].xaxis.set_major_formatter(formatter_time)
ax[0].set_xlim((0,0.0003))
ax[0].legend()
ax[1].plot(t,0.5 * Vpp_sim_signal * np.cos(2 * np.pi * sim_signal_freq * t),'red',label='MF Signal')
ax[1].yaxis.set_major_formatter(formatter_V)
ax[1].xaxis.set_major_formatter(formatter_time)
ax[1].set_xlim((0,0.0003))
ax[1].legend()
#ax[3].set_xscale('log')
ax[2].yaxis.set_major_formatter(formatter_db)
ax[2].xaxis.set_major_formatter(formatter_freq)
ax[2].plot(fft_frequencies,fft_amplitudes, color='blue')
ax[2].axvline(sim_gear_freq, linestyle=':',color='r', label='{} Tri'.format(formatter_freq(sim_gear_freq)),zorder=0)
ax[2].set_xlim((0,50000))
plt.tight_layout()
plt.show()